In [77]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [29]:
### use 2016 and 2020 data voter data sourced from boulder county, lets see if we can predict the 2020 election
### then, lets see if we can predict 2024 using the same model!
### best to use classifier, and classify precinct as voting for a given candidate based on previous voting history


def importVotingData():
    voterRegistrationSetOne = pd.read_csv('VotingData/VoterDetailsListPart1.csv', sep=',', low_memory=False)
    voterRegistrationSetTwo = pd.read_csv('VotingData/VoterDetailsListPart2.csv', sep=',', low_memory=False)
    return voterRegistrationSetOne, voterRegistrationSetTwo


def import2020VotingResultsByPrecinct():
    precinctResults = pd.read_csv('VotingData/2020BoulderCountyElectionResults.csv', sep=',', low_memory=False)
    return precinctResults

def cleanVoterRegistrationData(df = None):
    df = df[['VOTER_ID', 'VOTER_STATUS',  'BIRTH_YEAR', 'GENDER', 'PRECINCT_CODE', 'PARTY',]]
    df.PARTY.replace(['DEM', 'REP'], [0, 1], inplace=True)
    df.VOTER_STATUS.replace(['Active', 'Inactive'], [1, 0], inplace=True)
    df.GENDER.replace(['Male', 'Female', 'X' , 'Not Disclosed'], [1, 0, 0, 0], inplace=True)
    df = df[(df.PARTY == 0) | (df.PARTY == 1)]



    return df



    

In [30]:
voterRegistrationSetOne, voterRegistrationSetTwo = importVotingData()


In [31]:
voterRegistrationSetOne = cleanVoterRegistrationData(voterRegistrationSetOne)
voterRegistrationSetTwo = cleanVoterRegistrationData(voterRegistrationSetTwo)



C:\Users\william\AppData\Local\Temp\ipykernel_21808\3507401150.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.PARTY.replace(['DEM', 'REP'], [0, 1], inplace=True)
C:\Users\william\AppData\Local\Temp\ipykernel_21808\3507401150.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.VOTER_STATUS.replace(['Active', 'Inactive'], [1, 0], inplace=True)
C:\Users\william\AppData\Local\Temp\ipykernel_21808\3507401150.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [32]:
voterRegistrationSet = pd.concat([voterRegistrationSetOne, voterRegistrationSetTwo], ignore_index=True)

In [33]:
precinctResults = import2020VotingResultsByPrecinct()
voterRegistrationSet['PARTY'] = pd.to_numeric(voterRegistrationSet['PARTY'])

In [117]:
### - Reorganize column order for ease of access

#reorganizedColumns = ['VOTER_ID', 'VOTER_STATUS','GENDER','BIRTH_YEAR', 'PRECINCT_CODE', 'PARTY', ]
reorganizedColumns = ['VOTER_STATUS','GENDER', 'PARTY']
voterRegistrationSet = voterRegistrationSet.reindex(columns=reorganizedColumns)


In [118]:
print(voterRegistrationSet)

        VOTER_STATUS  GENDER  PARTY
0                  1       1      0
1                  0       0      0
2                  1       1      0
3                  1       1      0
4                  1       1      0
...              ...     ...    ...
127194             1       1      1
127195             0       1      0
127196             1       1      0
127197             1       0      0
127198             1       1      0

[127199 rows x 3 columns]


In [119]:
voterRegistrationSetNumpy = voterRegistrationSet.to_numpy()

In [132]:
voterRegistrationSet_train, voterRegistrationSet_test, voterParty_train, voterParty_test = train_test_split(voterRegistrationSetNumpy[:,0:2], voterRegistrationSetNumpy[:,2], test_size=0.10, train_size=0.10, random_state=10)
model = LogisticRegression(random_state=10)
model.fit(voterRegistrationSet_train, voterParty_train)
voterPredict = model.predict(voterRegistrationSet_test)
accuracy = accuracy_score(voterParty_test, voterPredict)
print(accuracy)

0.7857704402515723
